In [2]:
!pip install scikit-multilearn
!pip install peft
!pip install bitsandbytes
!pip install accelerate
!pip install wandb
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.4/89.4 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 8.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manyli

In [4]:
exit()

In [1]:
import os
import random
import functools
import csv
import numpy as np
import pandas as pd
import torch
import torch.nn.functional as F
from sklearn.metrics import f1_score
from skmultilearn.model_selection import iterative_train_test_split
from peft import PeftModel, PeftConfig
from peft import (
    LoraConfig,
    prepare_model_for_kbit_training,
    get_peft_model
)
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
    Trainer
)

In [2]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
# Method and Model Configuration
entity = "rstern"
retrain_from_checkpoint = True
model_config = "org_model"
checkpoint = "checkpoint-400"
author_label_only = False
augmented_data = False
experiment_name = "org_org_b"
debugg = False
# model name
model_name = 'mistralai/Mistral-7B-v0.1'

In [5]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False
if IN_COLAB:
  from google.colab import drive
  import sys
  drive.mount('/content/drive')
  sys.path.append('/content/drive/MyDrive/ucph/LP Project/test-quant-model') # If working in collab change this path
  path = '/content/drive/MyDrive/ucph/LP Project/test-quant-model/'
else:
  test_df_0 = pd.read_csv(f'test_df_0.csv')
  test_df_1 = pd.read_csv(f'test_df_1.csv')
  test_df_2 = pd.read_csv(f'test_df_2.csv')



In [6]:
test_df_0 = test_df_0.sample(frac=1, random_state=42)
test_df_1 = test_df_1.sample(frac=1, random_state=42)
test_df_2 = test_df_2.sample(frac=1, random_state=42)

In [7]:
def create_sequences(row):
  sequence = str(row["paragraph1"]) + "[LP2]" + str(row["paragraph2"])
  return sequence

test_df_0["input"] = test_df_0.apply(create_sequences, axis=1)
test_df_1["input"] = test_df_1.apply(create_sequences, axis=1)
test_df_2["input"] = test_df_2.apply(create_sequences, axis=1)

x_test_0 = test_df_0["input"].values
x_test_1 = test_df_1["input"].values
x_test_2 = test_df_2["input"].values

def create_multilabel(row):
  label_dataset = max(min(1, row["label_dataset"]), 0)
  multilabel = np.array([int(row["label_author"]), int(label_dataset)])
  return multilabel

def create_singlelabel(row):
  label = np.array([int(row["label_author"])])
  return label

if author_label_only:
  test_df_0["label"] = test_df_0.apply(create_singlelabel, axis=1)
  test_df_1["label"] = test_df_1.apply(create_singlelabel, axis=1)
  x_test_2["label"] = x_test_2.apply(create_singlelabel, axis=1)
  label_weights = [1]
else:
  test_df_0["label"] = test_df_0.apply(create_multilabel, axis=1)
  test_df_1["label"] = test_df_1.apply(create_multilabel, axis=1)
  test_df_2["label"] = test_df_2.apply(create_multilabel, axis=1)
  # weight author label heavier than topic change label
  label_weights = [2,1]


y_test_0 = test_df_0["label"].values
y_test_1 = test_df_1["label"].values
y_test_2 = test_df_2["label"].values
y_test_0 = np.stack(y_test_0)
y_test_1 = np.stack(y_test_1)
y_test_2 = np.stack(y_test_2)



In [8]:
from datasets import Dataset, DatasetDict

ds = DatasetDict({
    'test_0': Dataset.from_dict({'text': x_test_0, 'labels': y_test_0}),
    'test_1': Dataset.from_dict({'text': x_test_1, 'labels': y_test_1}),
    'test_2': Dataset.from_dict({'text': x_test_2, 'labels': y_test_2}),
})

In [9]:
def tokenize_examples(examples, tokenizer):
    tokenized_inputs = tokenizer(examples['text'])
    tokenized_inputs['labels'] = examples['labels']
    return tokenized_inputs

tokenizer = AutoTokenizer.from_pretrained(model_name)

# new tokens
new_tokens = ["[LP2]"]
new_tokens = set(new_tokens) - set(tokenizer.vocab.keys())
tokenizer.add_tokens(list(new_tokens))
tokenizer.pad_token = tokenizer.eos_token

tokenizer.pad_token = tokenizer.eos_token
tokenized_ds = ds.map(functools.partial(tokenize_examples, tokenizer=tokenizer), batched=True)
tokenized_ds = tokenized_ds.with_format('torch')

Map:   0%|          | 0/2236 [00:00<?, ? examples/s]

Map:   0%|          | 0/2236 [00:00<?, ? examples/s]

Map:   0%|          | 0/2236 [00:00<?, ? examples/s]

In [10]:
def collate_fn(batch, tokenizer):
    dict_keys = ['input_ids', 'attention_mask', 'labels']
    d = {k: [dic[k] for dic in batch] for k in dict_keys}
    d['input_ids'] = torch.nn.utils.rnn.pad_sequence(
        d['input_ids'], batch_first=True, padding_value=tokenizer.pad_token_id
    )
    d['attention_mask'] = torch.nn.utils.rnn.pad_sequence(
        d['attention_mask'], batch_first=True, padding_value=0
    )
    d['labels'] = torch.stack(d['labels'])
    return d

# define which metrics to compute for evaluation
def compute_metrics(p):
    predictions, labels = p
    f1_micro = f1_score(labels, predictions > 0, average = 'micro')
    f1_macro = f1_score(labels, predictions > 0, average = 'macro')
    f1_weighted = f1_score(labels, predictions > 0, average = 'weighted')
    return {
        'f1_micro': f1_micro,
        'f1_macro': f1_macro,
        'f1_weighted': f1_weighted
    }


# create custom trainer class to be able to pass label weights and calculate multilabel loss
class CustomTrainer(Trainer):

    def __init__(self, label_weights, **kwargs):
        super().__init__(**kwargs)
        self.label_weights = label_weights

    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.pop("labels")

        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")

        # compute custom loss
        loss = F.binary_cross_entropy_with_logits(logits, labels.to(torch.float32), pos_weight=self.label_weights)
        return (loss, outputs) if return_outputs else loss


In [12]:

config = PeftConfig.from_pretrained(f"{experiment_name}/{checkpoint}")
model = AutoModelForSequenceClassification.from_pretrained(config.base_model_name_or_path)
model.resize_token_embeddings(len(tokenizer))
lora_model = PeftModel.from_pretrained(model, f"{experiment_name}/{checkpoint}", is_trainable=True)

# there are many errors when pushing to the hub, as the embedding layer sizes mismatch.
# lora_model.push_to_hub("mistral-lp2-test-model", tokenizer=tokenizer, config=config)


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Some weights of MistralForSequenceClassification were not initialized from the model checkpoint at mistralai/Mistral-7B-v0.1 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
from torch.utils.data import DataLoader

# Create custom collate function
def collate_fn(batch):
    dict_keys = ['input_ids', 'attention_mask', 'labels']
    d = {k: [dic[k] for dic in batch] for k in dict_keys}
    d['input_ids'] = torch.nn.utils.rnn.pad_sequence(
        d['input_ids'], batch_first=True, padding_value=tokenizer.pad_token_id
    )
    d['attention_mask'] = torch.nn.utils.rnn.pad_sequence(
        d['attention_mask'], batch_first=True, padding_value=0
    )
    d['labels'] = torch.stack(d['labels'])
    return d

def evaluate(ds_name):
    # Create DataLoader
    test_loader = DataLoader(tokenized_ds[ds_name], batch_size=1, collate_fn=collate_fn)
    
    # Move model to CPU for evaluation
    lora_model.to('cpu')
    
    # Evaluation loop
    lora_model.eval()
    all_predictions = []
    all_labels = []
    
    with torch.no_grad():
        for batch in test_loader:
            input_ids = batch['input_ids']
            attention_mask = batch['attention_mask']
            labels = batch['labels']
    
            outputs = lora_model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits
            predictions = torch.sigmoid(logits)
            all_predictions.extend(predictions.numpy())
            all_labels.extend(labels.numpy())
    
    # Convert predictions and labels to numpy arrays
    all_predictions = np.array(all_predictions)
    all_labels = np.array(all_labels)
    
    # Compute metrics
    all_predictions_binary = (all_predictions > 0.5).astype(int)
    if author_label_only:
        f1_micro = f1_score(all_labels, all_predictions_binary, average='micro')
        f1_macro = f1_score(all_labels, all_predictions_binary, average='macro')
        f1_weighted = f1_score(all_labels, all_predictions_binary, average='weighted')
    else:
        f1_micro = f1_score(all_labels[:,0], all_predictions_binary[:,0], average='micro')
        f1_macro = f1_score(all_labels[:,0], all_predictions_binary[:,0], average='macro')
        f1_weighted = f1_score(all_labels[:,0], all_predictions_binary[:,0], average='weighted')
    
    metrics = {
        'f1_micro': f1_micro,
        'f1_macro': f1_macro,
        'f1_weighted': f1_weighted
    }
    
    print(ds_name, metrics)

In [14]:
evaluate('test_0')
evaluate('test_1')
evaluate('test_2')

test_0 {'f1_micro': 0.830948121645796, 'f1_macro': 0.6093944099378882, 'f1_weighted': 0.8519983555372838}
test_1 {'f1_micro': 0.7133273703041145, 'f1_macro': 0.6970032777805377, 'f1_weighted': 0.7083892536833022}
test_2 {'f1_micro': 0.5313059033989267, 'f1_macro': 0.523730471211169, 'f1_weighted': 0.5217963183121671}
